<font size=6><b> 재무정보 DataFrame화

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings(action='ignore')

#-------------------- 차트 관련 속성 (한글처리, 그리드) -----------
#plt.rc('font', family='NanumGothicOTF') # For MacOS
plt.rcParams['font.family']= 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
sns.set()

#-------------------- 주피터 , 출력결과 넓이 늘리기 ---------------
from IPython.core.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('max_colwidth', None)

import dart_fss as dart

In [2]:
# fn_target_year = 4  # 2022
# target_year = '2022'
# target_year2 = '2021'

In [3]:
fn_target_year = 1  # 2022
target_year = '2019'
target_year2 = '2018'

# 주가정보 Load

In [5]:
# 기업주가df = pd.read_csv('./datasets/찐최종기업주가.csv')
# 기업주가df = 기업주가df.set_index('날짜')
# 기업주가df = 기업주가df.sort_index()
# 기업주가df.tail()

In [6]:
# len(기업주가df.columns)

# 재무재표기간 로드

In [7]:
# 재무재표기간df = pd.read_csv('./datasets/찐기업별재무재표기간.csv')
# 재무재표기간df = 재무재표기간df.set_index('기업명')
# 재무재표기간df.head()

In [8]:
# len(재무재표기간df)

In [ ]:
# 종목list = list(기업주가df.columns.values)
# 종목list[:5]

In [9]:
# for i in range(len(종목list)):
#     if 종목list[i][-2:] =='.1':
#         print(종목list[i])
#         종목list[i] = 종목list[i][:-2]

# 손익계산서 (fnguide crawling) 로드

In [10]:
import os

filelist = os.listdir('./데이터/기업별 재무제표/재무제표 원본/balance_sheet')
filelist[:2]

['CMG제약.xlsx', 'HLB.xlsx']

In [ ]:


# f1 : 당기순이익 > 0 (흑자)
당기순이익list = []

# f2 : 영업현금흐름 > 0 (흑자) dart
# 영업현금흐름list = []

# f3 : 현재 ROA > 전년 ROA 
ROAlist = []
전년ROAlist = []

# f4: 영업현금흐름 > 순이익 dart

# f5: 현재 부채비율 < 전년부채비율
부채비율 = []
전년부채비율 = []

# f6: 유동비율 > 전년유동비율 dart
# 유동비율   

# f7: 당해 신규주식 발행 이력 (없으면 +1)
발행주식이력list = []

# f8: 매출총이익 > 전년 매출총이익
매출액list = []
전년매출액list = []

# f9: 자산회전율 > 전년 자산회전율
# 자산회전율       # 매출액/자산총계
자산회전율list = []
전년자산회전율list = []

for 종목 in 종목list:
    filename = ""
    for f in filelist:
        if f.find(종목) != -1:
            print(f)
            filename = f
            break
            
    if filename == '':
        당기순이익list.append(-1)
        ROAlist.append(-1)
        전년ROAlist.append(-1)
        부채비율.append(-1)
        전년부채비율.append(-1)
        발행주식이력list.append(-1)
        매출액list.append(-1)
        전년매출액list.append(-1)
        자산회전율list.append(-1)
        전년자산회전율list.append(-1)
    else:
        df = pd.read_csv(f'./datasets/손익계산서/{filename}')
        df = df.set_index('IFRS(연결)')
        
        당기순이익list.append(float(df.iloc[4, fn_target_year]))
        ROAlist.append(float(df.iloc[17, fn_target_year]))
        try:
            전년ROAlist.append(float(df.iloc[17, fn_target_year-1]))
        except:
            전년ROAlist.append(0)
            # print("ERROR ", df.iloc[17, fn_target_year-1])
            
        
        if df.iloc[13, fn_target_year] == '완전잠식':
            부채비율.append(10000)
        else:
            부채비율.append(float(df.iloc[13, fn_target_year]))
        
        if df.iloc[13, fn_target_year-1] == '완전잠식':
            전년부채비율.append(10000)
        else:
            전년부채비율.append(float(df.iloc[13, fn_target_year-1]))
        
        if df.iloc[24, fn_target_year] > df.iloc[24, fn_target_year-1]:
            발행주식이력list.append(1)
        else:
            발행주식이력list.append(0)
        
        매출액list.append(float(df.iloc[1, fn_target_year]))
        전년매출액list.append(float(df.iloc[1, fn_target_year-1]))
        자산회전율list.append(float(df.iloc[1, fn_target_year])/float(df.iloc[7, fn_target_year]))
        전년자산회전율list.append(float(df.iloc[1, fn_target_year-1])/float(df.iloc[7, fn_target_year-1]))

In [ ]:
ㄴㅇㅎㄴㅇㅎㅇㄶ

In [ ]:
for 종목 in 종목list:
    filename = ""
    for f in filelist:
        if f.find(종목) != -1:
            print(f)
            filename = f
            break
            

# Dart data 로드

In [ ]:
# f2 : 영업현금흐름 > 0 (흑자) dart
# 영업현금흐름list = []

# f4: 영업현금흐름 > 당기순이익 dart


영업활동현금흐름list = []

for sl in 종목list:
    fsdf1 =pd.read_excel(f"./datasets/재무제표/{sl}.xlsx",sheet_name='Data_cf',header=0,usecols= [2]+list(np.arange(8,42)),skiprows=[1,2])

    target_index = -1

    for i in range(1, len(fsdf1.columns)):
        if fsdf1.columns[i][:4] ==  target_year:
            target_index= i 
            break
        
    if target_index == -1:
        영업활동현금흐름list.append(-1)
    else:
        try:
            영업활동현금흐름idx = fsdf1[fsdf1['Unnamed: 2'].str.contains('영업활동현금흐름')].index.values[0]
            영업활동현금흐름 = fsdf1.iloc[영업활동현금흐름idx, target_index]
            영업활동현금흐름list.append(영업활동현금흐름)
        except:
            영업활동현금흐름list.append(-2)

In [ ]:
# f6: 유동비율 > 전년유동비율 dart
# 유동비율  = 유동자산/총자산

자산총계list = []
유동자산list = []

for sl in 종목list:
    fsdf1 =pd.read_excel(f"./datasets/재무제표/{sl}.xlsx",sheet_name='Data_bs',header=0,usecols= [2]+list(np.arange(8,42)),skiprows=[1,2])

    target_index = -1

    for i in range(1, len(fsdf1.columns)):
        if fsdf1.columns[i][:4] ==  target_year:
            target_index= i 
            break
        
    if target_index == -1:
        자산총계list.append(-1)
        유동자산list.append(-1)
    else:
        자산총계idx = fsdf1[fsdf1['Unnamed: 2'].str.contains('자산총계')].index.values[0]
        자산총계 = fsdf1.iloc[자산총계idx, target_index]
        자산총계list.append(자산총계)

        유동자산idx = fsdf1[fsdf1['Unnamed: 2'].str.contains('유동자산')].index.values[0]
        유동자산 = fsdf1.iloc[유동자산idx, target_index]
        유동자산list.append(유동자산)

In [ ]:
유동비율list = list(np.array(유동자산list)/np.array(자산총계list))

In [ ]:
자산총계list2 = []
유동자산list2 = []

for sl in 종목list:
    fsdf1 =pd.read_excel(f"./datasets/재무제표/{sl}.xlsx",sheet_name='Data_bs',header=0,usecols= [2]+list(np.arange(8,42)),skiprows=[1,2])

    target_index = -1

    for i in range(1, len(fsdf1.columns)):
        if fsdf1.columns[i][:4] ==  target_year2:
            target_index= i 
            break
        
    if target_index == -1:
        자산총계list2.append(-1)
        유동자산list2.append(-1)
    else:
        자산총계idx = fsdf1[fsdf1['Unnamed: 2'].str.contains('자산총계')].index.values[0]
        자산총계 = fsdf1.iloc[자산총계idx, target_index]
        자산총계list2.append(자산총계)

        유동자산idx = fsdf1[fsdf1['Unnamed: 2'].str.contains('유동자산')].index.values[0]
        유동자산 = fsdf1.iloc[유동자산idx, target_index]
        유동자산list2.append(유동자산)

In [ ]:
전년유동비율list = list(np.array(유동자산list2)/np.array(자산총계list2))

# DataFrame 처리

In [ ]:
# f1 : 당기순이익 > 0 (흑자)
# f2 : 영업현금흐름 > 0 (흑자) dart
# f3 : 현재 ROA > 전년 ROA 
# f4: 영업현금흐름 > 순이익 dart
# f5: 현재 부채비율 < 전년부채비율
# f6: 유동비율 > 전년유동비율 dart
# f7: 당해 신규주식 발행 이력 (없으면 +1)
# f8: 매출총이익 > 전년 매출총이익
# f9: 자산회전율 > 전년 자산회전율

fsdic = {'당기순이익':당기순이익list, 
         '영업활동현금흐름':영업활동현금흐름list,
         'ROA':ROAlist, '전년ROA':전년ROAlist, 
         '부채비율':부채비율, '전년부채비율':전년부채비율, 
         '유동비율':유동비율list,'전년유동비율':전년유동비율list,
         '발행주식이력':발행주식이력list, 
         '매출액':매출액list,'전년매출액':전년매출액list, 
         '자산회전율':자산회전율list, '전년자산회전율':전년자산회전율list}

fsdic = pd.DataFrame(fsdic)

fsdic.index = 종목list
fsdic.head(10)

In [ ]:
fsdic.to_csv(f'./datasets/재무제표df{target_year}.csv', encoding='cp949')

In [11]:
df = pd.read_excel('./데이터/기업별 재무제표/재무제표 원본/balance_sheet/강스템바이오텍.xlsx','Data_bs')
df

,Unnamed: 0,"[D210000] Statement of financial position, current/non-current - Consolidated financial statements (Unit: KRW)",Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,20221231,20211231,20201231,20191231,20181231,20171231,20161231,20151231
0,NaN,concept_id,label_ko,label_en,class0,class1,class2,class3,"('연결재무제표',)","('연결재무제표',)","('연결재무제표',)","('연결재무제표',)","('연결재무제표',)","('연결재무제표',)","('연결재무제표',)","('연결재무제표',)"
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,ifrs-full_CurrentAssets,유동자산,Current assets,재무상태표 [abstract],자산 [abstract],유동자산,NaN,45906520168,65355113853,44750598843,55408094623,48992789806,25246011811,37966496235,18896589947
3,1.0,ifrs-full_CashAndCashEquivalents,현금및현금성자산,Cash and cash equivalents,재무상태표 [abstract],자산 [abstract],유동자산,현금및현금성자산,4680560978,7541589035,9968671983,7680863204,3221945619,3913815075,1928056618,6276858790
4,2.0,dart_ShortTermDepositsNotClassifiedAsCashEquivalents,단기금융상품,Short-term deposits not classified as cash equivalents,재무상태표 [abstract],자산 [abstract],유동자산,단기금융상품,31008492797,39017642733,24800649651,7000000000,8280000000,8116000000,22400000000,10600000000
5,3.0,ifrs-full_CurrentFinancialAssetsAtFairValueThroughProfitOrLossMandatorilyMeasuredAtFairValue,당기손익-공정가치측정금융자산,"Current financial assets at fair value through profit or loss, mandatorily measured at fair value",재무상태표 [abstract],자산 [abstract],유동자산,당기손익-공정가치측정금융자산,988823300,10026219177,3164040000,33543082216,27170261818,NaN,0,1016282000
6,4.0,ifrs-full_TradeAndOtherCurrentReceivables,매출채권 및 기타채권,Trade and other current receivables,재무상태표 [abstract],자산 [abstract],유동자산,매출채권 및 기타채권,5779305377,5006039110,4269604709,3499797386,5928134432,3996839840,586690505,471975569
7,5.0,ifrs-full_CurrentTaxAssets,당기법인세자산,Current tax assets,재무상태표 [abstract],자산 [abstract],유동자산,당기법인세자산,133195720,59990280,125933130,120263580,49435960,50535960,30774400,8651910
8,6.0,ifrs-full_CurrentContractAssets,계약자산,Current contract assets,재무상태표 [abstract],자산 [abstract],유동자산,계약자산,656206292,321906104,NaN,NaN,NaN,NaN,NaN,NaN
9,7.0,ifrs-full_Inventories,재고자산,Inventories,재무상태표 [abstract],자산 [abstract],유동자산,재고자산,467150029,763768689,354858965,266393477,1918575351,1725224948,582807610,334912787


# F Score 계산

In [ ]:
tyear = "2019"
df = pd.read_csv(f'./datasets/재무제표df/재무제표df{tyear}.csv', encoding='cp949')
df = df.set_index('Unnamed: 0')
df.head()

In [ ]:
df.info()

In [ ]:
# f1 : 당기순이익 > 0 (흑자)

# f2 : 영업현금흐름 > 0 (흑자) dart
# 영업현금흐름list = []

# f3 : 현재 ROA > 전년 ROA 

# f4: 영업현금흐름 > 순이익 dart

# f5: 현재 부채비율 < 전년부채비율

# f6: 유동비율 > 전년유동비율 dart
# 유동비율   

# f7: 당해 신규주식 발행 이력 (없으면 +1)

# f8: 매출총이익 > 전년 매출총이익

# f9: 자산회전율 > 전년 자산회전율
# 자산회전율       # 매출액/자산총계

df['f1'] = np.where(df['당기순이익']>0, 1, 0)
df['f2'] = np.where(df['영업활동현금흐름']>0, 1, 0)
df['f3'] = np.where(df['ROA']>df['전년ROA'], 1, 0)
df['f4'] = np.where(df['영업활동현금흐름']>df['당기순이익'], 1, 0)
df['f5'] = np.where(df['부채비율']>df['전년부채비율'], 1, 0)
df['f6'] = np.where(df['유동비율']>df['전년유동비율'], 1, 0)
df['f7'] = df['발행주식이력']
df['f8'] = np.where(df['매출액']>df['전년매출액'], 1, 0)
df['f9'] = np.where(df['자산회전율']>df['전년자산회전율'], 1, 0)

df.head()

In [ ]:
df.to_csv(f'./datasets/재무제표df/재무제표df{tyear}.csv', encoding='cp949')